In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import zipfile

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') > .99:
            print("\nReached 99% accuracy so cancelling training!")
            self.model.stop_training = True

mnist = tf.keras.datasets.mnist

(X_train_images, y_train_labels), (X_test_images, y_test_labels) = mnist.load_data()

X_train_images = X_train_images.reshape(60000,28,28,1)
X_train_images = X_train_images/255.0
X_test_images = X_test_images.reshape(10000,28,28,1)
X_test_images = X_test_images/255.0

In [ ]:
callbacks = myCallback()
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (5,5), activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(3,3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# model fitting
history = model.fit(
    X_train_images,
    y_train_labels,
    epochs=15,
    callbacks=[callbacks]
)

Epoch 1/15
1875/1875 [==============================] - 44s 23ms/step - loss: 0.1320 - accuracy: 0.9596
Epoch 2/15
1875/1875 [==============================] - 43s 23ms/step - loss: 0.0455 - accuracy: 0.9864
Epoch 3/15
1875/1875 [==============================] - 43s 23ms/step - loss: 0.0324 - accuracy: 0.9898
Epoch 4/15
1875/1875 [==============================] - 42s 23ms/step - loss: 0.0234 - accuracy: 0.9925

Reached 99% accuracy so cancelling training!


In [ ]:
# model fitting
print(history.epoch, history.history['accuracy'][-1])

[0, 1, 2, 3] 0.9924833178520203


In [ ]:
model.evaluate(X_test_images, y_test_labels)

313/313 [==============================] - 2s 7ms/step - loss: 0.0426 - accuracy: 0.9867


[0.042566895484924316, 0.9866999983787537]

In [ ]:
mnist = tf.keras.datasets.mnist
(X_train_images, y_train_labels), (X_test_images, y_test_labels) = mnist.load_data()

X_train_images = X_train_images.reshape(60000,28,28,1)
X_test_images = X_test_images.reshape(10000,28,28,1)

train_datagen = ImageDataGenerator(rescale=1.0/255.0,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

print(X_train_images.shape)
print(X_test_images.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') > .85:
            print("\nReached 85% accuracy so cancelling training!")
            self.model.stop_training = True

callbacks = myCallback()
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (5,5), activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(3,3),
    tf.keras.layers.Conv2D(64, (5,5), activation='relu'),
    tf.keras.layers.MaxPooling2D(3,3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

train_generator = train_datagen.flow(X_train_images, y_train_labels, batch_size=64, seed=0)
validation_generator = validation_datagen.flow(X_test_images, y_test_labels, batch_size=64, seed=0)

In [ ]:
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    epochs=50,
                    callbacks=[callbacks],
                    verbose=1)

Epoch 1/50
938/938 [==============================] - 32s 34ms/step - loss: 1.2256 - accuracy: 0.5800 - val_loss: 0.3410 - val_accuracy: 0.8958
Epoch 2/50
938/938 [==============================] - 31s 33ms/step - loss: 0.7350 - accuracy: 0.7562 - val_loss: 0.2617 - val_accuracy: 0.9205
Epoch 3/50
938/938 [==============================] - 31s 33ms/step - loss: 0.6189 - accuracy: 0.7967 - val_loss: 0.2188 - val_accuracy: 0.9324
Epoch 4/50
938/938 [==============================] - 31s 33ms/step - loss: 0.5621 - accuracy: 0.8149 - val_loss: 0.1877 - val_accuracy: 0.9409
Epoch 5/50
938/938 [==============================] - 31s 33ms/step - loss: 0.5087 - accuracy: 0.8350 - val_loss: 0.2121 - val_accuracy: 0.9323
Epoch 6/50
938/938 [==============================] - 31s 33ms/step - loss: 0.4765 - accuracy: 0.8465 - val_loss: 0.1581 - val_accuracy: 0.9478
Epoch 7/50
938/938 [==============================] - 32s 34ms/step - loss: 0.4574 - accuracy: 0.8521 - val_loss: 0.1602 - val_accuracy:

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

--2021-10-04 05:31:24--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.121.128, 142.250.103.128, 108.177.120.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.121.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M  90.2MB/s    in 0.7s    

2021-10-04 05:31:25 (90.2 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [ ]:
zip_ref = zipfile.ZipFile("/tmp/cats_and_dogs_filtered.zip", 'r')
zip_ref.extractall("tmp/")
zip_ref.close()

# Define our example directories and files
base_dir = 'tmp/cats_and_dogs_filtered'

train_dir = os.path.join( base_dir, 'train')
validation_dir = os.path.join( base_dir, 'validation')


train_cats_dir = os.path.join(train_dir, 'cats') # Directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs') # Directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats') # Directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')# Directory with our validation dog pictures

In [57]:
train_datagen = ImageDataGenerator(rescale=1.0/255.0,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(train_dir, batch_size=24, class_mode='binary', target_size=(150,150))
validation_generator = validation_datagen.flow_from_directory(validation_dir, batch_size=24, class_mode='binary', target_size=(150,150))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [59]:
pre_trained_model = InceptionV3(input_shape=(150,150,3),
                                include_top=False,
                                weights='imagenet')

for layer in pre_trained_model.layers:
  layer.trainable = False

pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d_204 (Conv2D)             (None, 74, 74, 32)   864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_188 (BatchN (None, 74, 74, 32)   96          conv2d_204[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, 74, 74, 32)   0           batch_normalization_188[0][0]    
_______________________________________________________________________________________

In [60]:
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(.2)(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(pre_trained_model.input, x)
model.compile(optimizer=RMSprop(learning_rate=.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [65]:
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 40,
            epochs = 2,
            validation_steps = 20,
            callbacks=[callbacks],
            verbose=1)

Epoch 1/2
40/40 [==============================] - 11s 281ms/step - loss: 0.1702 - accuracy: 0.9396 - val_loss: 0.1062 - val_accuracy: 0.9563

Reached 85% accuracy so cancelling training!
